In [9]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests
import pandas as pd
import time
from datetime import datetime

# --- PARÁMETROS ESPECÍFICOS PARA TU TAREA ---
GENRES = ['funk']
YEARS = [2010, 2024] 

In [10]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import os
from dotenv import load_dotenv

load_dotenv() # Carga las variables del archivo .env

SPOTIPY_CLIENT_ID = os.getenv('SPOTIPY_CLIENT_ID')
SPOTIPY_CLIENT_SECRET = os.getenv('SPOTIPY_CLIENT_SECRET')
LASTFM_API_KEY = os.getenv('LASTFM_API_KEY')

In [12]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# 1. Años para iterar
YEARS = [2010, 2024]

# 2. Lista de géneros relacionados con funk (Python, no texto)
funk_related_genres = [
    # Funk clásico
    "funk",
    "classic funk",
    "deep funk",
    "funky",
    "funky breaks",
    "p-funk",
    "funk rock",
    "funk metal",
    "afro funk",
    "jazz funk",
    "disco funk",
    "boogie",
    "old school funk",

    # Soul / neo soul
    "soul",
    "classic soul",
    "neo soul",
    "contemporary r&b",
    "motown",
    "funk soul",
    
    # Funk house / dance
    "funk house",
    "disco house",
    "funky house",
    "deep funk house",
    "electro funk",
    "dance funk",
    "groove house"
]

# 3. Términos optimizados para búsqueda (puedes usar directamente la lista anterior)
query_terms = funk_related_genres

# 4. Inicializa el cliente de Spotify
SPOTIPY_CLIENT_ID = 'bc4bfe3fd40e4866bf24b434a7ed6d41'
SPOTIPY_CLIENT_SECRET = 'f2754b20f21948c69e02a7509d75f701'

try:
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
        client_id=SPOTIPY_CLIENT_ID,
        client_secret=SPOTIPY_CLIENT_SECRET
    ))
    print("✅ Conexión a Spotify exitosa.")
except Exception as e:
    print(f"❌ Error de conexión a Spotify: {e}")
    sp = None


✅ Conexión a Spotify exitosa.


In [13]:
# NUEVA CELDA: Comprobación de autenticación
print(type(sp)) 

<class 'spotipy.client.Spotify'>


In [23]:
def extract_spotify_data_funk_ampliada(
    sp,
    years,
    query_terms,
    max_total_songs=500,
    max_songs_per_year=250
): 
    """
    Itera por AÑO y por cada TÉRMINO de género.
    - Limita el total global de canciones a max_total_songs.
    - Limita el máximo por año a max_songs_per_year.
    """
    spotify_data = []
    total_songs = 0  # contador global

    for year in years:
        year_range = f'{year}-{year}'
        songs_in_year = 0  # contador por año
        print(f"\n--- INICIANDO EXTRACCIÓN PARA EL AÑO {year} ---")

        for term in query_terms:
            # Si ya se alcanzó el límite global o el del año, paramos
            if total_songs >= max_total_songs:
                print("\n⚠️ Límite global de canciones alcanzado. Deteniendo extracción.")
                return pd.DataFrame(spotify_data)

            if songs_in_year >= max_songs_per_year:
                print(f"⚠️ Límite de canciones para el año {year} alcanzado ({songs_in_year}). Pasando al siguiente año.")
                break

            query = f'genre:"{term}" year:{year_range}'
            print(f"   -> Buscando con término: {term}")

            try:
                results = sp.search(q=query, type='track', limit=50)
                total_resultados_api = results['tracks']['total'] if results and 'tracks' in results else 0
                print(f"   -> Spotify reporta {total_resultados_api} resultados para '{term}'.")

                while results and total_songs < max_total_songs and songs_in_year < max_songs_per_year:
                    for track in results['tracks']['items']:
                        track_id = track['id']

                        # Evitar duplicados
                        if any(d.get('ID_Spotify') == track_id for d in spotify_data):
                            continue

                        if total_songs >= max_total_songs or songs_in_year >= max_songs_per_year:
                            break

                        data_row = {
                            'Artista': track['artists'][0]['name'],
                            'Género musical': f'funk (Origen: {term})',
                            'Tipo': 'Canción',
                            'Nombre': track['name'],
                            'Año de lanzamiento': year,
                            'ID_Spotify': track_id,
                            'ID_Album': track['album']['id']
                        }
                        spotify_data.append(data_row)
                        total_songs += 1
                        songs_in_year += 1

                    # Paginación
                    if results['tracks']['next'] and total_songs < max_total_songs and songs_in_year < max_songs_per_year:
                        results = sp.next(results['tracks'])
                    else:
                        results = None

                    time.sleep(0.5)

                print(f"      -> Acumulado año {year}: {songs_in_year} | Total global: {total_songs}")

            except Exception as e:
                print(f"   -> Error al buscar con término '{term}': {e}")
                time.sleep(2)

    return pd.DataFrame(spotify_data)


# --- LLAMADA ---
if sp:
    df_spotify_raw = extract_spotify_data_funk_ampliada(
        sp,
        YEARS,
        query_terms,
        max_total_songs=500,    # máximo total
        max_songs_per_year=250  # máximo por año
    )
    print("\n--- RESULTADO FINAL DE FUNK (ESTRATEGIA ROBUSTA) ---")
    print(f"Total de registros extraídos: {len(df_spotify_raw)}")
    print(df_spotify_raw.head(10))



--- INICIANDO EXTRACCIÓN PARA EL AÑO 2010 ---
   -> Buscando con término: funk
   -> Spotify reporta 85 resultados para 'funk'.
      -> Acumulado año 2010: 85 | Total global: 85
   -> Buscando con término: classic funk
   -> Spotify reporta 5 resultados para 'classic funk'.
      -> Acumulado año 2010: 85 | Total global: 85
   -> Buscando con término: deep funk
   -> Spotify reporta 10 resultados para 'deep funk'.
      -> Acumulado año 2010: 85 | Total global: 85
   -> Buscando con término: funky
   -> Spotify reporta 9 resultados para 'funky'.
      -> Acumulado año 2010: 85 | Total global: 85
   -> Buscando con término: funky breaks
   -> Spotify reporta 7 resultados para 'funky breaks'.
      -> Acumulado año 2010: 85 | Total global: 85
   -> Buscando con término: p-funk
   -> Spotify reporta 21 resultados para 'p-funk'.
      -> Acumulado año 2010: 85 | Total global: 85
   -> Buscando con término: funk rock
   -> Spotify reporta 5 resultados para 'funk rock'.
      -> Acumulado 

In [24]:
import time
import pandas as pd

In [20]:
if sp:
    # Suponiendo que df_spotify_raw tiene la columna 'ID_Album'
    album_ids = df_spotify_raw['ID_Album'].tolist()

    df_albums = extract_album_details(sp, album_ids)
    print("\n--- RESUMEN ÁLBUMES ---")
    print(df_albums.head(10))
    print(f"\n🔢 Número total de álbumes únicos: {len(df_albums)}")



--- INICIANDO EXTRACCIÓN DE METADATOS DE 324 ÁLBUMES ---
✅ Extracción de 324 álbumes completada.

--- RESUMEN ÁLBUMES ---
                 ID_Album                                       Nombre_Album  \
0  2dfmUaxPxLyFhNyUOwjSUq                                Brown's Ferry Blues   
1  1SWqQVknpugMUrzMGzaWJr                                          Hot Stone   
2  1aF9xLIt4BTbWvuJgFHrZb                                 Día del Amigo 2024   
3  4HJhkccVXPlS3qx3v2i07C                                Berts Apple Crumble   
4  2iLrKttv0jpnTD6vCl3poM  The Ever Wonderful Ted Taylor: Okeh Uptown Sou...   
5  33PqghN6IyqZ0bwXG56fon                                     Stimmung - Gut   
6  66OGSssNXg4nHeEpAMZEhq                                      Junko Partner   
7  5gtaS4I65T2W5bIyFD2SEU                                           Melodies   
8  7zZjo73u7A7ST6OpeXhGoI                       ...do ya wanna dance wit me?   
9  7Cj5CRcDiaJzEcPoK22vDL                                           Halfpipe 

In [22]:
# Total de canciones
print("Canciones (filas en df_spotify_raw):", len(df_spotify_raw))

# Total de álbumes únicos (antes de llamar a la función)
print("Álbumes únicos en df_spotify_raw:", df_spotify_raw['ID_Album'].nunique())

# Total de álbumes con metadatos obtenidos
print("Álbumes con detalles devueltos por la función:", len(df_albums))


Canciones (filas en df_spotify_raw): 500
Álbumes únicos en df_spotify_raw: 324
Álbumes con detalles devueltos por la función: 324
